# Topic Selection and Postprocessing

For label annotation

In [1]:
%matplotlib inline
import pickle
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import re
import math

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%pylab inline
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from tqdm._tqdm_notebook import tqdm_notebook

from constants import *
from utils import load, init_logging, tprint, TopicsLoader, Unlemmatizer
from topic_reranking import Reranker

tqdm_notebook.pandas()

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.options.display.max_columns = 80
pd.options.display.max_rows = 2000
pd.options.display.precision = 3

In [3]:
phra = load('phrases', 'lemmap')
wikt = load('wikt', 'lemmap')

Loading ../data/preprocessed/dewiki_phrases_lemmatization_map.pickle


-----

##### Joining Topics / Labels / Scores

In [4]:
#pfile_t = join(TPX_PATH, 'PoliticalSpeeches_topics_ep20.csv')
ofile_t = join(TPX_PATH, 'OnlineParticipation_topics_ep20.csv')
#ptopics = pd.read_csv(pfile_t, index_col=[0, 1, 2, 3, 4], header=0).reset_index('metric', drop=True)
otopics = pd.read_csv(ofile_t, index_col=[0, 1, 2, 3, 4], header=0).reset_index('metric', drop=True)
otopics

term0               term1               term2               term3                    term4                term5                 term6                   term7               term8                   term9
dataset             param_id nb_topics topic_idx                                                                                                                                                                                                                    
OnlineParticipation e42      100       0           Bad_Godesberg                Lage        Hauptbahnhof               Stadt            Pflasterstein                  Gag               Rathaus                  Berlin          Innenstadt                    City
                                       1                  Arbeit             Aufgabe            Toilette            Personal                   Steuer               Kosten          Finanzierung            Gesellschaft       Allgemeinheit                   Stadt
                                       2               Vorschlag      Bürgerhaushalt           Redaktion              Bürger                    Thema           Verwaltung     Bezirksvertretung                   Grund               Stadt             Möglichkeit
                                       3                   Seite              Brücke        Kreisverkehr               Beuel                       KM                Umweg              Fahrspur             Rodenkirche  Verkehrsteilnehmer                 Bereich
                                       4                 Familie              Gruppe                Köln                Geld               Stadt_Köln                 Mann           Einrichtung                   Hilfe         Blücherpark          Lärmschutzwand
                                       5                 Fahrrad      Fahrradständer          Verwaltung         Bevölkerung                   Rahmen                 Jahr              Zuschuss         Bürgervorschlag            Mehrheit          Fahrradverkehr
                                       6                   Stadt                Land           Wuppertal            Bücherei                     Bund               Bürger                   NRW               Tourismus                Geld                  Aktion
                                       7                 Nutzung          Grünfläche       Veranstaltung            Rheinaue                    Stadt                  LED              Bebauung            Veranstalter            Einfluss                Akademie
                                       8                 Viertel            Maßnahme            Freizeit       Neugestaltung                 Ärgernis              Passant                  Glas      Bordsteinabsenkung                Jahr  Einbahnstraßenregelung
                                       9            Fahrradwrack          Zustimmung          Blumenberg          Radständer               Chorweiler         Verlängerung                Pfütze                    Reif      Fahrradständer                      BV
                                       10             Moderation                Gruß           Kommentar                Dank                    Hallo              Hinweis                Dialog              Diskussion             Aussage              Teilnehmer
                                       11                  Miete                Luft              Bürger           Vermieter                    Stadt               Bezirk                Mieter                    Jahr                Köln                  Objekt
                                       12             Innenstadt            Geschäft                StVO            Kalkberg                    Weide             Standort            Frankenbad                  Besuch             Auswahl         Rollstuhlfahrer
                                       13             Schlagloch                Zone               Regen               Abend                   Z

In [5]:
#pfile_l = join(TPX_PATH, 'PoliticalSpeeches_labels_ep20.csv')
ofile_l = join(TPX_PATH, 'OnlineParticipation_labels_ep20.csv')
#plabels = (
#    pd
#    .read_csv(pfile_l, index_col=[0, 1, 2, 3, 4], header=0)
#    .drop('label_method', axis=1)
#    .reset_index('metric', drop=True)
#    .applymap(lambda x: eval(x)[0])
#)
olabels = (
    pd
    .read_csv(ofile_l, index_col=[0, 1, 2, 3, 4], header=0)
    .drop('label_method', axis=1)
    .reset_index('metric', drop=True)
    .applymap(lambda x: eval(x)[0])
)
olabels

label0                                             label1                              label2                                          label3                                             label4                         label5                                     label6                                             label7                                             label8                                          label9                     label10                                         label11                          label12                                         label13                           label14                           label15                                            label16                                            label17                                label18                                 label19
dataset             param_id nb_topics topic_idx                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
OnlineParticipation e42      100       0                                             Altstadt                                historisch_Altstadt                          Stadtmitte                                      Innenstadt                                 historisch_Fünfeck                      Kernstadt               Seevorstadt-Ost/Große_Garten                                      Alt-Godesberg                                           Vorstadt                                     Altstadt_II                 alt_Rathaus                                       Weststadt                 Bahnhofsvorplatz                                      Marktplatz             statistisch_Stadtteil                          Südstadt                                     Rathausviertel                                        neu_Rathaus                              S-Bahnhof                          Bismarckstraße
                                       1                                      öffentlich_Hand                                         Aufwendung                    Mitgliedsbeitrag                                   Wohnungswesen                                       Unterhaltung       Bahnversicherungsanstalt                              Dienstleister                                         Verwaltung                        kalkulatorisch_Abschreibung                                   Infrastruktur                    Zuschuss                                      Raumkosten       mittelbar_Staatsverwaltung                             öffentlich_Haushalt                    Kommunalabgabe           Landeswohlfahrtsverband                                   Generaldirektion                                        Anschaffung               übertragen_Wirkungskreis                             Verpflegung
                                       2                                      Kommunalpolitik                                       Bürgerschaft                            Stadtrat                                   Mandatsträger                            Vertretungskörperschaft                  Stadtteilbüro                      düsseldorfer_Stadtrat  sozialdemokratisch_Gemeinschaft_für_Kommunalpo...                                    Bürgerw

In [6]:
#pfile_s = join(TPX_PATH, 'PoliticalSpeeches_scores_ep20_X.csv')
ofile_s = join(TPX_PATH, 'OnlineParticipation_scores_ep20_X.csv')
#pscores = pd.read_csv(pfile_s, index_col=[0, 1, 2, 3, 4], header=0).reset_index('metric', drop=True)
oscores = pd.read_csv(ofile_s, index_col=[0, 1, 2, 3, 4], header=0).reset_index('metric', drop=True)
oscores

c_npmi_wikt  pairwise_similarity_ftx
dataset             param_id nb_topics topic_idx                                      
OnlineParticipation e42      100       0           -2.266e-02                    0.342
                                       1            2.746e-02                    0.378
                                       2            2.396e-02                    0.375
                                       3            3.981e-04                    0.328
                                       4           -6.237e-02                    0.275
                                       5           -2.908e-02                    0.323
                                       6            3.274e-02                    0.339
                                       7           -1.890e-02                    0.316
                                       8           -4.189e-02                    0.300
                                       9           -8.703e-02                    0.252
                                       10           5.358e-02                    0.336
                                       11           2.359e-02                    0.315
                                       12          -1.211e-01                    0.261
                                       13          -7.810e-03                    0.255
                                       14          -8.113e-03                    0.316
                                       15           5.240e-02                    0.389
                                       16           4.774e-02                    0.339
                                       17          -1.153e-02                    0.320
                                       18          -7.458e-02                    0.276
                                       19           1.272e-01                    0.439
                                       20          -7.832e-02                    0.303
                                       21          -1.451e-02                    0.284
                                       22          -3.070e-02                    0.310
                                       23           4.719e-02                    0.338
                                       24          -1.204e-01                    0.250
                                       25           2.322e-02                    0.325
                                       26          -8.209e-02                    0.325
                                       27          -7.380e-02                    0.270
                                       28          -6.117e-03                    0.314
                                       29           1.225e-03                    0.361
                                       30          -6.599e-02                    0.290
                                       31           3.855e-03                    0.328
                                       32          -3.225e-02                    0.318
                                       33          -8.247e-02                    0.287
                                       34           3.069e-03                    0.344
                                       35           1.061e-02                    0.334
                                       36          -4.867e-02                    0.310
                                       37           2.241e-01                    0.527
                                       38           3.629e-02                    0.357
                                       39          -1.788e-02                    0.292
                                       40          -8.488e-02                    0.289
                                       41          -6.310e-02                    0.303
                                       42           1.746e-01                    0.512
                                       43          -1.387e-01                    0.226
                                       44       

In [8]:
metrics = ['c_npmi_wikt', 'pairwise_similarity_ftx']

def combine_simple(topics, labels, scores, min_regular_terms=6, min_cnpmi=0, min_pairsim=0.3):
    scores['rank_cnpmi'] = scores.c_npmi_wikt.rank()
    scores['rank_pairsim'] = scores.pairwise_similarity_ftx.rank()
    scores['rank_mean'] = (scores.rank_cnpmi + scores.rank_pairsim) / 2
    col_mean = scores[metrics].mean()
    col_stdv = scores[metrics].std()
    print(col_mean)
    scores_norm = (scores[metrics] - col_mean) / col_stdv
    scores_norm = scores_norm.rename(columns=lambda x: x + '_norm')
    scores_norm['avg'] = scores_norm.mean(axis=1)
    scores = scores.join(scores_norm)
    scores['avg_rank'] = scores.avg.rank()
    print('average difference per topic between the mean of the individual ranks and the rank of the normalized mean:', (scores.rank_mean - scores.avg_rank).abs().sum() / len(scores))
    # an advantage of ranking by taking the normalized mean: minimal chance of equal ranks -> disadvantage: metric is not comparable to other datasets anymore -> use metrics as a filter

    df = pd.concat([topics, labels, scores], axis=1)
    df['count_regular_terms'] = df.loc[:, 'term0':'term9'].applymap(lambda x: x in wikt.index).sum(axis=1)
    df = df.query('count_regular_terms >= @min_regular_terms and c_npmi_wikt >= @min_cnpmi and pairwise_similarity_ftx >= @min_pairsim')
    print(len(df))
    df = df.sort_values('avg_rank', ascending=False).loc[:, 'term0':'label19']
    return df

#dfp = combine_simple(ptopics, plabels, pscores, min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfo = combine_simple(otopics, olabels, oscores, min_regular_terms=7, min_cnpmi=-0.05, min_pairsim=0.3)
dfo

c_npmi_wikt               -0.021
pairwise_similarity_ftx    0.319
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.03
81


term0               term1               term2               term3                    term4                term5                 term6                   term7            term8                   term9                                       label0                                             label1                              label2                                          label3                                  label4                         label5                                     label6                                             label7                                             label8                                          label9                     label10                                         label11                          label12                                         label13                           label14                           label15                                            label16                                 label17                                label18                                 label19
dataset             param_id nb_topics topic_idx                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
OnlineParticipation e42      100       94              Radfahrer              Straße          Autofahrer       Fahrradstraße                 Richtung        Fahrradfahrer            Fahrradweg                    Auto        Fußgänger                  Radweg                                   Radverkehr                                      Fahrradstraße                          zweispurig                              Verkehrsteilnehmer                               Veloroute                Kraftfahrstraße                                  Radfahrer                                      Einbahnstraße                                     Straßenverkehr                                    Unterführung                   Parkstand                                      Ravel-Netz           Beschleunigungsstreife                                    Kreisverkehr                   Fußgängerbrücke                            Gehweg                                         Bahnkörper                            Gegenverkehr                             Wohnstraße                         Milseburgradweg
                                       37                    Bus                Bahn               Linie                ÖPNV                  Strecke           Verbindung           Haltestelle                    Stau         Buslinie              Busbahnhof                                     Stadtbus                                         Busbahnhof                    Durchmesserlinie                   öffentlich_Personennahverkehr                          Nahverkehrszug                     Nahverkehr                                 Schnellbus                              Schienenersatzverkehr                                            Umsteig                                      Endbahnhof           Verknüpfungspunkt                                     Ha

In [10]:
args = ['e42', 100]
metrics = ['c_npmi_wikt', 'pairwise_similarity_ftx']

def combine(dataset, min_regular_terms=6, min_cnpmi=0, min_pairsim=0.3):
    topics = load(dataset, 'topics', *args)
    labels_w2v = load(dataset, 'labels', *args, 'w2v', 'minimal').rename(columns=lambda x: x + '_w2v').reset_index(level='label_method', drop=True)
    labels_ftx = load(dataset, 'labels', *args, 'ftx', 'minimal').rename(columns=lambda x: x + '_ftx').reset_index(level='label_method', drop=True)
    scores = load(dataset, 'scores', *args, 'ftx', 'minimal')[metrics]

    scores['rank_cnpmi'] = scores.c_npmi_wikt.rank()
    scores['rank_pairsim'] = scores.pairwise_similarity_ftx.rank()
    scores['rank_mean'] = (scores.rank_cnpmi + scores.rank_pairsim) / 2
    col_mean = scores[metrics].mean()
    col_stdv = scores[metrics].std()
    print(col_mean)
    scores_norm = (scores[metrics] - col_mean) / col_stdv
    scores_norm = scores_norm.rename(columns=lambda x: x + '_norm')
    scores_norm['avg'] = scores_norm.mean(axis=1)
    scores = scores.join(scores_norm)
    scores['avg_rank'] = scores.avg.rank()
    print('average difference per topic between the mean of the individual ranks and the rank of the normalized mean:', (scores.rank_mean - scores.avg_rank).abs().sum() / len(scores))
    # an advantage of ranking by taking the normalized mean: minimal chance of equal ranks -> disadvantage: metric is not comparable to other datasets anymore -> use metrics as a filter

    df = pd.concat([topics, labels_w2v, labels_ftx, scores], axis=1)
    df['count_regular_terms'] = df.loc[:, 'term0':'term9'].applymap(lambda x: x in wikt.index).sum(axis=1)
    df = df.query('count_regular_terms >= @min_regular_terms and c_npmi_wikt >= @min_cnpmi and pairwise_similarity_ftx >= @min_pairsim')
    print(len(df))
    df = df.sort_values('avg_rank', ascending=False).loc[:, 'term0':'label19_w2v'].rename(columns=lambda x: x.replace('_w2v', ''))
    return df

In [23]:
dfd = combine('dewac', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfn = combine('n', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
#dfs = combine('s', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
#dfo = combine('o', min_regular_terms=6, min_cnpmi=-0.1, min_pairsim=0.3)
dfp = combine('p', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)

Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.123
pairwise_similarity_ftx    0.457
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.12
72
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/news_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/news_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_label-candidates.csv
Readin

In [24]:
df = pd.concat([dfd, dfn, dfp, dfo]).reset_index(level=['param_id', 'nb_topics'], drop=True).sort_index()
df

term0             term1  \
dataset             topic_idx                                         
OnlineParticipation 0               Bad_Godesberg              Lage   
                    1                      Arbeit           Aufgabe   
                    2                   Vorschlag    Bürgerhaushalt   
                    5                     Fahrrad    Fahrradständer   
                    6                       Stadt              Land   
                    7                     Nutzung        Grünfläche   
                    10                 Moderation              Gruß   
                    11                      Miete              Luft   
                    14                     Museum          Eindruck   
                    15                   Haushalt         Einkommen   
                    19                     Radweg         Radfahrer   
                    23                       Euro              Jahr   
                    25                      Stadt          Einnahme   
                    28                       Müll              Bank   
                    29                    Schritt             Stadt   
                    31                  Parkplatz          Anwohner   
                    32                       Hund         Kontrolle   
                    34                        EUR          Leistung   
                    35                      Frage      Unterführung   
                    36                  Sanierung             Umbau   
                    37                        Bus              Bahn   
                    38                    Beitrag              Dank   
                    42                       Kind            Schule   
                    50                        Uhr               2_.   
                    53               Verbesserung             Mitte   
                    55                Unternehmen               Job   
                    57                     Bürger              Hand   
                    60                     Schild            Ticket   
                    61                      Firma       Straßenbahn   
                    66                     Poller           Verkehr   
                    68              Einbahnstraße      Falschparker   
                    70                    Politik           Mädchen   
                    72                    Meinung           Zukunft   
                    74                        Bad        Grundstück   
                    79                       Auto           Fahrrad   
                    80                Mitarbeiter               Amt   
                    81                     Fläche              Frau   
                    84                        Weg       Beleuchtung   
                    89                        Rad           Bahnhof   
                    90                  Radfahrer         Fußgänger   
                    94                  Radfahrer            Straße   
                    95                       Haus           Bildung   
                    96                   Richtung            Radweg   
                    97                     Gehweg              Park   
                    98                       Baum              Höhe   
PoliticalSpeeches   0                Wissenschaft         Forschung   
                    5                       Musik             Abend   
                    7                    Religion            Glaube   
                    11                Universität        Hochschule   
                    14                     Schule           Schüler   
                    18                      Krise              Bank   
                    19                         EU  europäisch_Union   
                    21             Zusammenarbeit             China   
                    22                 Erinnerung        Geschichte   
                    23                       Bund            Kultur 

In [25]:
file = join(DATA_BASE, 'topics_labels_X.csv')
df.to_csv(file)

In [26]:
ul = Unlemmatizer()

Loading ../data/preprocessed/dewiki_phrases_lemmatization_map.pickle


In [27]:
df.loc[:, 'term0':'term9']

term0             term1  \
dataset             topic_idx                                         
OnlineParticipation 0               Bad_Godesberg              Lage   
                    1                      Arbeit           Aufgabe   
                    2                   Vorschlag    Bürgerhaushalt   
                    5                     Fahrrad    Fahrradständer   
                    6                       Stadt              Land   
                    7                     Nutzung        Grünfläche   
                    10                 Moderation              Gruß   
                    11                      Miete              Luft   
                    14                     Museum          Eindruck   
                    15                   Haushalt         Einkommen   
                    19                     Radweg         Radfahrer   
                    23                       Euro              Jahr   
                    25                      Stadt          Einnahme   
                    28                       Müll              Bank   
                    29                    Schritt             Stadt   
                    31                  Parkplatz          Anwohner   
                    32                       Hund         Kontrolle   
                    34                        EUR          Leistung   
                    35                      Frage      Unterführung   
                    36                  Sanierung             Umbau   
                    37                        Bus              Bahn   
                    38                    Beitrag              Dank   
                    42                       Kind            Schule   
                    50                        Uhr               2_.   
                    53               Verbesserung             Mitte   
                    55                Unternehmen               Job   
                    57                     Bürger              Hand   
                    60                     Schild            Ticket   
                    61                      Firma       Straßenbahn   
                    66                     Poller           Verkehr   
                    68              Einbahnstraße      Falschparker   
                    70                    Politik           Mädchen   
                    72                    Meinung           Zukunft   
                    74                        Bad        Grundstück   
                    79                       Auto           Fahrrad   
                    80                Mitarbeiter               Amt   
                    81                     Fläche              Frau   
                    84                        Weg       Beleuchtung   
                    89                        Rad           Bahnhof   
                    90                  Radfahrer         Fußgänger   
                    94                  Radfahrer            Straße   
                    95                       Haus           Bildung   
                    96                   Richtung            Radweg   
                    97                     Gehweg              Park   
                    98                       Baum              Höhe   
PoliticalSpeeches   0                Wissenschaft         Forschung   
                    5                       Musik             Abend   
                    7                    Religion            Glaube   
                    11                Universität        Hochschule   
                    14                     Schule           Schüler   
                    18                      Krise              Bank   
                    19                         EU  europäisch_Union   
                    21             Zusammenarbeit             China   
                    22                 Erinnerung        Geschichte   
                    23                       Bund            Kultur 

In [28]:
file = join(DATA_BASE, 'topics_unlem_X.csv')
df_topics_unlem = ul.unlemmatize_topics(df.loc[:, 'term0':'term9'])
df_topics_unlem.to_csv(file)
df_topics_unlem

Loading ../data/preprocessed/OnlineParticipation_lemmatization_map.pickle
    Bad_Godesberg -> Bad Godesberg
    Parkplatz -> Parkplätze
    Hund -> Hunde
    Bus -> Busse
    Kind -> Kinder
    Auto -> Autos
    Fläche -> Flächen
    Baum -> Bäume
    Bad_Godesberg -> Bad Godesberg
    Parkplatz -> Parkplätze
    Hund -> Hunde
    Bus -> Busse
    Kind -> Kinder
    Auto -> Autos
    Fläche -> Flächen
    Baum -> Bäume
    Grünfläche -> Grünflächen
    Gruß -> Grüße
    Jahr -> Jahren
    Einnahme -> Einnahmen
    Bank -> Bänke
    Kontrolle -> Kontrollen
    Leistung -> Leistungen
    Schule -> Schulen
    2_. -> 2.
    Frau -> Frauen
    Toilette -> Toiletten
    Veranstaltung -> Veranstaltungen
    Information -> Informationen
unkown phrase Stadt_Bonn
Stadt
token in wikt
Stadt
Bonn
    Stadt_Bonn -> Stadt Bonn
    Wohnung -> Wohnungen
    Entscheidung -> Entscheidungen
    Minute -> Minuten
    Radstreife -> Radstreifen
    Bürgerin -> Bürgerinnen
    Stellplatz -> Stellplätze
    

    Kunde -> Kunden
    Arzt -> Ärzte
    Kind -> Kinder
    Foto -> Fotos
    Mitglied -> Mitglieder
    Schuh -> Schuhe
    Mensch -> Menschen
    Protest -> Proteste
    Flüchtling -> Flüchtlinge
    Milliarde -> Milliarden
    Mensch -> Menschen
    Mensch -> Menschen
    Reis -> Reise
    Patient -> Patienten
    Milliarde -> Milliarden
    Fahrzeug -> Fahrzeuge
    Vorwurf -> Vorwürfe
    Tonne -> Tonnen
    Stunde -> Stunden
    Gast -> Gäste
    Farbe -> Farben
    Demonstrant -> Demonstranten
    Punkt -> Punkte
    Beamter -> Beamten
    Gerät -> Geräte
    Mensch -> Menschen
    Million -> Millionen
    Behörde -> Behörden
    Angeklagte -> Angeklagten
    Beschäftigte -> Beschäftigten
    Stimme -> Stimmen
    Folge -> Folgen
    Ei -> Eis
    Haar -> Haare
    Immobilie -> Immobilien
    Deutsch -> Deutschen
    Aktie -> Aktien
    Kunde -> Kunden
    Mensch -> Menschen
    Tote -> Toten
    Polizist -> Polizisten
    Mensch -> Menschen
    Produkt -> Produkte
    Tourist 

term0              term1  \
dataset             topic_idx                                            
OnlineParticipation 0                 Bad Godesberg               Lage   
                    1                        Arbeit            Aufgabe   
                    2                     Vorschlag     Bürgerhaushalt   
                    5                       Fahrrad     Fahrradständer   
                    6                         Stadt               Land   
                    7                       Nutzung        Grünflächen   
                    10                   Moderation              Grüße   
                    11                        Miete               Luft   
                    14                       Museum           Eindruck   
                    15                     Haushalt          Einkommen   
                    19                       Radweg          Radfahrer   
                    23                         Euro             Jahren   
                    25                        Stadt          Einnahmen   
                    28                         Müll              Bänke   
                    29                      Schritt              Stadt   
                    31                   Parkplätze           Anwohner   
                    32                        Hunde         Kontrollen   
                    34                          EUR         Leistungen   
                    35                        Frage       Unterführung   
                    36                    Sanierung              Umbau   
                    37                        Busse               Bahn   
                    38                      Beitrag               Dank   
                    42                       Kinder            Schulen   
                    50                          Uhr                 2.   
                    53                 Verbesserung              Mitte   
                    55                  Unternehmen                Job   
                    57                       Bürger               Hand   
                    60                       Schild             Ticket   
                    61                        Firma        Straßenbahn   
                    66                       Poller            Verkehr   
                    68                Einbahnstraße       Falschparker   
                    70                      Politik            Mädchen   
                    72                      Meinung            Zukunft   
                    74                          Bad         Grundstück   
                    79                        Autos            Fahrrad   
                    80                  Mitarbeiter                Amt   
                    81                      Flächen             Frauen   
                    84                          Weg        Beleuchtung   
                    89                          Rad            Bahnhof   
                    90                    Radfahrer          Fußgänger   
                    94                    Radfahrer             Straße   
                    95                         Haus            Bildung   
                    96                     Richtung             Radweg   
                    97                       Gehweg               Park   
                    98                        Bäume               Höhe   
PoliticalSpeeches   0                  Wissenschaft          Forschung   
                    5                         Musik              Abend   
                    7                      Religion            Glauben   
                    11                  Universität        Hochschulen   
                    14                      Schulen            Schüler   
                    18                        Krise             Banken   
                    19                           EU  Europäische Union   
                    21               Zusammenarbei

In [29]:
file = join(DATA_BASE, 'labels_unlem_X.csv')
df_labels_unlem = ul.unlemmatize_labels(df.loc[:, 'label0':'label19'])
df_labels_unlem.to_csv(file)
df_labels_unlem

    öffentlich_Hand -> Öffentliche Hand
    Verkehrsverbund_Pforzheim-Enzkreis -> Verkehrsverbund Pforzheim-Enzkreis
    Städte-_und_Gemeindebund_Nordrhein-Westfale -> Städte- und Gemeindebund Nordrhein-Westfalen
    sozial_Einrichtung -> Soziale Einrichtung
    Hundegesetz -> Hundegesetze
    deutsch_Fußballmeisterschaft_der_A-Junior -> Deutsche Fußballmeisterschaft der A-Junioren
    Hollerstaude -> Hollerstauden
    One_Equity_Partner -> One Equity Partners
    Beschleunigungsstreife -> Beschleunigungsstreifen
    zweispurig -> Zweispurig
    übertragen_Wirkungskreis -> Übertragener Wirkungskreis
    Beschleunigungsstreife -> Beschleunigungsstreifen
    Aspekt_Salzburg -> Aspekte Salzburg
    Arbeitskreis_selbständig_Kultur-Institut -> Arbeitskreis selbständiger Kultur-Institute
    frei_Entfaltung_der_Persönlichkeit -> Freie Entfaltung der Persönlichkeit
    Regierungskonsultation -> Regierungskonsultationen
    israelitisch_Kultusgemeinde -> Israelitische Kultusgemeinde
    zeitge

    Verbrechen_gegen_der_Menschlichkeit -> Verbrechen gegen die Menschlichkeit
    demokratisch_Gesellschaft -> Demokratische Gesellschaft
    industriell_Gemeinschaftsforschung -> Industrielle Gemeinschaftsforschung
    frei_Entfaltung_der_Persönlichkeit -> Freie Entfaltung der Persönlichkeit
    interkulturell_Dialog -> Interkultureller Dialog
    Un-Sonderbeauftragte -> UN-Sonderbeauftragter
    politisch_Ikonographie -> Politische Ikonographie
    politisch_Philosophie -> Politische Philosophie
    Copenhag_International_Film_Festival -> Copenhagen International Film Festival
    Arbeitskreis_selbständig_Kultur-Institut -> Arbeitskreis selbständiger Kultur-Institute
    Un-Sonderbeauftragte -> UN-Sonderbeauftragter
    beruflich_Handlungsfähigkeit -> Berufliche Handlungsfähigkeit
    Wolhyniendeutsch -> Wolhyniendeutsche
    politisch_Partei -> Politische Partei
    Prinzip_der_begrenzt_Einzelermächtigung -> Prinzip der begrenzten Einzelermächtigung
    romantisch_Komödie -> Romant

    schweizerisch_Flüchtlingshilfe -> Schweizerische Flüchtlingshilfe
    Prinzip_der_begrenzt_Einzelermächtigung -> Prinzip der begrenzten Einzelermächtigung
    historisch_Tatsache -> Historische Tatsachen
    deutsch_Bank -> Deutsche Bank
    digital_Kultur -> Digitale Kultur
    Reh -> Rehe
    Röstzwiebel -> Röstzwiebeln
    Pankreasenzym -> Pankreasenzyme
    pädagogisch_Hochschule -> Pädagogische Hochschule
    rot_Karte -> Rote Karte
    effektiv_Temperatur -> Effektive Temperatur
    alt_Haus -> Altes Haus
    nichts -> Nichts
    westfälisch_Wilhelms-Universität -> Westfälische Wilhelms-Universität
    Vergleichende_Sprachwissenschaft -> Vergleichende Sprachwissenschaft
    inner_Medizin -> Innere Medizin
    Yum_!_Brand -> Yum ! Brands
    Ermittlungsakt -> Ermittlungsakte
    Umts-Modem -> UMTS-Modem
    deutsch_Pflegerat -> Deutscher Pflegerat
    europäisch_Union -> Europäische Union
    Toggo_plus -> Toggo plus
    Landhaus_Scherrer -> Landhaus Scherrer
    Burgfestspiel

    Farn -> Farne
    zweite_Weltkrieg -> Zweiter Weltkrieg
    Die_Sonne -> Die Sonne
    Sonnenstern -> Sonnensterne
    sofortig_Beschwerde -> Sofortige Beschwerde
    Platzierungsrund -> Platzierungsrunde
    Angewandte_Kunst -> Angewandte Kunst
    Hauptstraße_34 -> Hauptstraße 34
    Haus_für_Kind -> Haus für Kinder
    Guilherand-Grang -> Guilherand-Granges
    Saint_Louis_Art_Museum -> Saint Louis Art Museum
    kosmisch_Staub -> Kosmischer Staub
    Die_Mutter -> Die Mutter
    Altria_Group -> Altria Group
    CIA -> Cia
    Maude_Adam -> Maude Adams
    Joch_Bendel -> Jochen Bendel
    Klaus_Höhne -> Klaus Höhne
    irakisch_Regierung -> Irakische Regierung
    historisch_Tatsache -> Historische Tatsachen
    islamisch_Emirat_Afghanistan -> Islamisches Emirat Afghanistan
    klein_Haus -> Kleines Haus
    Immobilienfond -> Immobilienfonds
    Full_Force -> Full Force
    häuslich_Gewalt -> Häusliche Gewalt
    Das_Erste -> Das Erste
    BIP -> Bip
    Nexus_5 -> Nexus 5
    i

    immer -> Immer
    öffentlich_Haushalt -> Öffentlicher Haushalt
    sozialdemokratisch_Gemeinschaft_für_Kommunalpolitik -> Sozialdemokratische Gemeinschaft für Kommunalpolitik
    Markthalle_München -> Markthallen München
    frankfurter_Alleenring -> Frankfurter Alleenring
    Sonntagabend -> SonntagAbend
    Vorbehalt_der_Gesetz -> Vorbehalt des Gesetzes
    zweispurig -> Zweispurig
    Ökumenische_Theologie -> Ökumenische Theologie
    hessisch_Filmförderung -> Hessische Filmförderung
    Finanz -> Finanzen
    polnisch_Aufstand -> Polnischer Aufstand
    Familienbund_der_Katholik -> Familienbund der Katholiken
    territorial_Integrität -> Territoriale Integrität
    hessisch_Filmförderung -> Hessische Filmförderung
    Forschung_und_Lehre -> Forschung und Lehre
    polnisch_Aufstand -> Polnischer Aufstand
    irakisch_Regierung -> Irakische Regierung
    Green_Economy -> Green Economy
    deutsch_historisch_Institut -> Deutsches Historisches Institut
    Il_Messaggero -> Il Me

label0  \
dataset             topic_idx                                                  
OnlineParticipation 0                                               Altstadt   
                    1                                       Öffentliche Hand   
                    2                                        Kommunalpolitik   
                    5                     Verkehrsverbund Pforzheim-Enzkreis   
                    6           Städte- und Gemeindebund Nordrhein-Westfalen   
                    7                                             Freifläche   
                    10                                             Bemerkung   
                    11                                         Baugrundstück   
                    14                                             Offenheit   
                    15                                            Aufwendung   
                    19                                          Unterführung   
                    23                                                   Mio   
                    25                                   Soziale Einrichtung   
                    28                                            Freifläche   
                    29                                         Privattheater   
                    31                                            Freifläche   
                    32                                          Hundegesetze   
                    34                                            Aufwendung   
                    35                                       Fußgängerbrücke   
                    36                                         Lückenschluss   
                    37                                              Stadtbus   
                    38                                                 Frage   
                    42                                           Schulklasse   
                    50          Deutsche Fußballmeisterschaft der A-Junioren   
                    53                                         Hollerstauden   
                    55                                   One Equity Partners   
                    57                                       Rechtsgrundlage   
                    60                                              Stadtbus   
                    61                                           Fabrikhalle   
                    66                                          Unterführung   
                    68                               Beschleunigungsstreifen   
                    70                                       Kommunalpolitik   
                    72                                Kornelimünster/Walheim   
                    74                                         Baugrundstück   
                    79                                            Zweispurig   
                    80                            Übertragener Wirkungskreis   
                    81                                            Freifläche   
                    84                                             Kernstadt   
                    89                                           Bahnstation   
                    90                                            Radverkehr   
                    94                                            Radverkehr   
                    95                                              Lesesaal   
                    96                               Beschleunigungsstreifen   
                    97                                             Parkstand   
                    98                                              Zeltdach   
PoliticalSpeeches   0                                   Naturwissenschaftler   
                    5                                       Aspekte Salzburg   
                    7                                               Religion   
                    11                                           Studi

In [15]:
dfd = combine('dewac', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfn = combine('n', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfp = combine('p', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfo = combine_simple(otopics, olabels, oscores, min_regular_terms=7, min_cnpmi=-0.05, min_pairsim=0.3)

Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.123
pairwise_similarity_ftx    0.457
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.12
72
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/news_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/news_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_label-candidates.csv
Readin

In [16]:
dfd

term0           term1            term2         term3         term4         term5              term6            term7                term8             term9                label0                            label1                                      label2                               label3                           label4                         label5                             label6                           label7                                   label8                                   label9                             label10                                    label11                      label12                             label13                     label14                           label15                                         label16                             label17                                      label18                                            label19
dataset param_id nb_topics topic_idx                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
dewac   e42      100       36               Montag         Freitag       Donnerstag      Mittwoch      Dienstag           Tag              Monat          Sonntag                Woche           Samstag            Donnerstag                          Dienstag                                     Freitag                              Samstag                       Wochenende                       Mittwoch                            Sonntag                     Sonntagabend                                 Feiertag                                Vormittag                             Werktag                              Tulpensonntag                    Wochentag                               Woche                heute_mittag             gut_Morgen_Österreich                                      Nachmittag                      Adventssonntag                           Hallo_Niedersachse                                             Termin
                           62         Arbeitnehmer     Arbeitgeber     Beschäftigte       Betrieb  Gewerkschaft     Kündigung        Arbeitszeit      Betriebsrat    Arbeitsverhältnis      Arbeitsplatz          Beschäftigte                       Belegschaft                                Arbeitnehmer                    Arbeitsverhältnis                     Tarifvertrag                   Arbeitskampf           Beschäftigungsverhältnis                      Arbeitgeber                              Betriebsrat                         Massenentlassung                        Arbeitsrecht                             Arbeitsvertrag          Tarifvertragspartei                   Freizeitausgleich                Tarifbindung                  Kündigungsschutz                                Tarifverhandlung                   Gesamtbetriebsrat                       Arbeitnehmervertretung                                 Firmentarifvertrag
                           56         Studierender         Studium          Student      Semester   Studiengang       Seminar         Hochschule        Vorlesung                 Kurs              Fach           Studiengang                       Studienfach                                     